In [5]:
pip install ipynb langchain langchain_community pypdf openai bs4 langchain-deepseek

Note: you may need to restart the kernel to use updated packages.


# RAG Basic Architecture

Below is the basic architecture of a RAG (Retrieval-Augmented Generation) system:

![RAG Basic Architecture](../resources/RAG/1_RAG_BASIC.png)

This diagram illustrates the core components and flow of a basic RAG system:
1. Document ingestion and chunking
2. Vector embedding and storage
3. Retrieval process
4. Generation with context

In [6]:
CHROMA_PATH = "chroma_rag1"


In [7]:
%load_ext autoreload
%autoreload 2
from dbUtils import DBUtils
dbUtils = DBUtils()
dbUtils.clear_database(CHROMA_PATH)
   
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Indexing

## Overview
Indexing is a crucial step in RAG systems where documents are processed, chunked, and stored in a vector database for efficient retrieval. This process transforms raw documents into searchable vectors that can be quickly accessed during the retrieval phase.

### Key Components:
1. Document Loading
   - PDF, Text, Web and other document format support
   - Metadata extraction
   - Character encoding handling

2. Text Chunking
   - Document segmentation strategies
   - Chunk size optimization
   - Overlap configuration

3. Embedding Generation
   - Text-to-vector transformation
   - Embedding model selection
   - Dimensionality considerations

4. Vector Storage
   - Vector database integration
   - Index optimization
   - Metadata management

## Process Flow
1. Load documents from various sources
2. Split documents into manageable chunks
3. Generate embeddings for each chunk

![RAG Components Detail](../resources/RAG/1_RAG_BASIC_2.png)

In [8]:
from pdfImportService import pdf_content_into_documents, format_docs

pdf_content_into_documents("../data/BeigeBook_20250115.pdf", CHROMA_PATH)



USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/easonyin/Project/github/AIJourney/RAG/dbUtils.py:39: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/easonyin/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


find chuck ../data/BeigeBook_20250115.pdf:0
find chuck ../data/BeigeBook_20250115.pdf:1
find chuck ../data/BeigeBook_20250115.pdf:2
find chuck ../data/BeigeBook_20250115.pdf:3
find chuck ../data/BeigeBook_20250115.pdf:4
find chuck ../data/BeigeBook_20250115.pdf:5
find chuck ../data/BeigeBook_20250115.pdf:6
find chuck ../data/BeigeBook_20250115.pdf:7
find chuck ../data/BeigeBook_20250115.pdf:8
find chuck ../data/BeigeBook_20250115.pdf:9
find chuck ../data/BeigeBook_20250115.pdf:10
find chuck ../data/BeigeBook_20250115.pdf:11
find chuck ../data/BeigeBook_20250115.pdf:12
find chuck ../data/BeigeBook_20250115.pdf:13
find chuck ../data/BeigeBook_20250115.pdf:14
find chuck ../data/BeigeBook_20250115.pdf:15
find chuck ../data/BeigeBook_20250115.pdf:16
find chuck ../data/BeigeBook_20250115.pdf:17
find chuck ../data/BeigeBook_20250115.pdf:18
find chuck ../data/BeigeBook_20250115.pdf:19
find chuck ../data/BeigeBook_20250115.pdf:20
find chuck ../data/BeigeBook_20250115.pdf:21
find chuck ../data/B

[Document(metadata={'source': '../data/BeigeBook_20250115.pdf', 'page': 0, 'id': '../data/BeigeBook_20250115.pdf:0:0'}, page_content='The Beige BookSummary of Commentary onCurrent Economic Conditions byFederal Reserve DistrictJanuary 2025\nFEDERAL RESERVE SYSTEM'),
 Document(metadata={'source': '../data/BeigeBook_20250115.pdf', 'page': 1, 'id': '../data/BeigeBook_20250115.pdf:1:0'}, page_content='ContentsAbout This Publication....................................................................................................iiNational Summary...........................................................................................................1Federal Reserve Bank of Boston..................................................................................5Federal Reserve Bank of New York..............................................................................8Federal Reserve Bank of Philadelphia.......................................................................12Federal Res

# Retrieval

## Overview
This section demonstrates how to retrieve relevant documents from our vector store using similarity search.

### Key Components:
- Query processing
- Vector similarity search
- Document retrieval

## Implementation

In [9]:
def retrieve_answers(question):
    db = dbUtils.get_chroma(CHROMA_PATH)
    results = db.similarity_search_with_relevance_scores(question, k=5)
    return results

In [10]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv() 


True

In [11]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [12]:
from langchain.prompts import ChatPromptTemplate
import pprint
query_text = "summarize current economy status"
retrieved_docs = retrieve_answers(query_text)

# retriever = dbUtils.get_chroma().as_retriever(search_kwargs={"k": 1})

# retrieved_docs = retriever.get_relevant_documents(query_text)
pprint.pprint(retrieved_docs)


[(Document(metadata={'id': '../data/BeigeBook_20250115.pdf:0:0', 'page': 0, 'source': '../data/BeigeBook_20250115.pdf'}, page_content='The Beige BookSummary of Commentary onCurrent Economic Conditions byFederal Reserve DistrictJanuary 2025\nFEDERAL RESERVE SYSTEM'),
  0.7183950957101538),
 (Document(metadata={'id': '../data/BeigeBook_20250115.pdf:6:0', 'page': 6, 'source': '../data/BeigeBook_20250115.pdf'}, page_content='RichmondThe regional economy continued to grow modestly this cycle. Consumer spending and travel grewmoderately over the holiday shopping and travel season. Financial services firms reported modestgrowth while demand for nonfinancial services was subdued. Manufacturing activity slowed slightlythis cycle and port activity was mixed. Employment levels were unchanged, and price growth wasmoderate.AtlantaEconomic activity in the Sixth District expanded modestly. Employment was steady and wagesgrew slowly. Input costs and prices increased modestly. Consumer spending grew mo

# Generation

## Overview
The generation phase is the final step in the RAG pipeline where retrieved context is combined with the user's query to produce accurate, contextually relevant responses. This phase leverages Large Language Models (LLMs) to synthesize information from retrieved documents and generate coherent answers.

### Key Components:
1. Context Integration
   - Retrieved document handling
   - Context window management
   - Document relevance scoring

2. Prompt Engineering
   - Template design
   - Context formatting
   - System instruction optimization

3. LLM Integration
   - Model selection and configuration
   - Temperature and sampling settings
   - Response formatting

4. Output Processing
   - Response validation
   - Source attribution
   - Answer refinement

## Process Flow
1. Context Preparation
   - Merge retrieved documents
   - Format context for LLM input
   - Apply relevance scoring

2. Prompt Construction
   - Combine user query with context
   - Apply prompt template
   - Add system instructions

3. Response Generation
   - Send to LLM
   - Process streaming responses
   - Handle token limits

4. Post-Processing
   - Validate response quality
   - Add source citations
   - Format final output

## Implementation
Below we'll implement the generation pipeline using LangChain and OpenAI's API.

![RAG Components Detail](../resources/RAG/1_RAG_BASIC_3.png)

![RAG LLM](../resources/RAG/1_RAG_BASIC_4.png)

In [13]:
def format_docs(docs):
    return "\n\n---\n\n".join([doc[0].page_content for doc in docs])

In [17]:
import os


In [18]:
load_dotenv()
DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')

In [19]:
context_text = format_docs(retrieved_docs)

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
client = OpenAI(api_key=os.getenv('DEEPSEEK_API_KEY'), base_url="https://api.deepseek.com")
stream = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides accurate information based on the given context."},
            {"role": "user", "content": prompt}],
        stream=False,
       
        temperature=1.3
    )

    

print(stream.choices[0].message.content)

The current economic status, as summarized from the Beige Book, indicates modest growth across various Federal Reserve Districts, with some regional variations:

1. **Consumer Spending and Travel**: Consumer spending grew moderately, particularly during the holiday shopping and travel season. Travel and tourism activity also expanded moderately.

2. **Employment and Wages**: Employment levels were generally steady, with modest wage growth reported in most districts. Some firms plan to maintain or slightly reduce staffing levels in 2025, while others anticipate modest hiring for growth. Wage increases are expected to align with pre-pandemic growth rates.

3. **Prices and Input Costs**: Prices and input costs increased modestly, with insurance and labor being the primary cost concerns for firms. Some firms plan to pass on cost increases to employees or final prices.

4. **Manufacturing**: Manufacturing activity slowed or declined slightly in most districts, although some contacts remain 

In [ ]:

from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import PromptTemplate

# Initialize the LLM by providing the model name and the temperature
retrieved_docs = retrieve_answers(query_text)
print(format_docs(retrieved_docs))

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


# Create the prompt template
prompt = PromptTemplate.from_template(
    """Answer the question based only on the following context:
    Context: {context}
    Question: {question}
    Answer: """
)

# Create the chain

chain = prompt | llm | StrOutputParser()
response = chain.invoke(
    {
        "context": format_docs(retrieved_docs),
        "question": "What are the current economic trends?"
    }
)

    
# response = rag_chain.invoke("What are the current economic trends?")
print(f"Answer: {response}")

The Beige BookSummary of Commentary onCurrent Economic Conditions byFederal Reserve DistrictJanuary 2025
FEDERAL RESERVE SYSTEM

---

RichmondThe regional economy continued to grow modestly this cycle. Consumer spending and travel grewmoderately over the holiday shopping and travel season. Financial services firms reported modestgrowth while demand for nonfinancial services was subdued. Manufacturing activity slowed slightlythis cycle and port activity was mixed. Employment levels were unchanged, and price growth wasmoderate.AtlantaEconomic activity in the Sixth District expanded modestly. Employment was steady and wagesgrew slowly. Input costs and prices increased modestly. Consumer spending grew moderately.Demand for housing increased modestly. Transportation activity grew slightly. Loan growth wasmoderate. Manufacturing declined slowly. Energy activity grew modestly, but agriculturedemand fell.ChicagoEconomic activity increased slightly. Consumer spending increased modestly; employm

Generation